<a href="https://colab.research.google.com/github/carlosaragao/PLN/blob/main/Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
artigo_treino = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/teste.csv")

In [ ]:
!python -m spacy download pt_core_news_sm

2023-12-20 22:54:47.269026: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 22:54:47.269102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 22:54:47.270700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 22:54:47.280395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 22:54:48.643355: W tensorflow/compiler/tf2

In [ ]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [ ]:
texto = "Adoro a cidade de caldas novas!"
doc = nlp(texto)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc.ents

(Adoro, caldas novas)

In [ ]:
doc[1].is_stop

True

In [ ]:
doc[1].is_alpha

True

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)

In [ ]:
def trata_textos(doc):
  tokens_validos = []


  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)


  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

In [ ]:
texto = "Adoro a 342342 #$@#@#$ cidade de caldas novas!"
doc = nlp(texto)
trata_textos(doc)

Resultado para o documento: Adoro cidade caldas


'Adoro cidade caldas'

In [ ]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        n_process = -1)]

In [ ]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})

In [ ]:
titulos_tratados.head()

,titulo
0,pai cariocas lúcia veríssimo vence festival
1,governo aumentar multa empresas assinam carteira
2,dias cantareira volta ficar acima
3,militares egípcios mortos atentado península s...
4,ataque triplo mata dezenas chade


In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [ ]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

76969
76969


In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [ ]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [ ]:
w2v_modelo.corpus_count

76969

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
        print('Loss após a época {}: {}'.format(self.epoch, loss))
    else:
        print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss


In [ ]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples = w2v_modelo.corpus_count,
                 epochs = 30,
                 compute_loss = True,
                 callbacks=[callback()])

Loss após a época 0: 128024.8046875
Loss após a época 1: 132844.4140625
Loss após a época 2: 111690.34375
Loss após a época 3: 113595.65625
Loss após a época 4: 114956.28125
Loss após a época 5: 109776.9375
Loss após a época 6: 124906.0
Loss após a época 7: 108823.0
Loss após a época 8: 93432.75
Loss após a época 9: 94198.3125
Loss após a época 10: 80607.5
Loss após a época 11: 79012.5
Loss após a época 12: 81198.625
Loss após a época 13: 89049.0
Loss após a época 14: 81453.625
Loss após a época 15: 80215.5
Loss após a época 16: 78417.125
Loss após a época 17: 67593.625
Loss após a época 18: 71154.5
Loss após a época 19: 78073.25
Loss após a época 20: 64399.375
Loss após a época 21: 71019.0
Loss após a época 22: 65394.375
Loss após a época 23: 72630.0
Loss após a época 24: 57091.5
Loss após a época 25: 67096.25
Loss após a época 26: 60068.25
Loss após a época 27: 61648.0
Loss após a época 28: 60507.25
Loss após a época 29: 56692.5


(13208782, 14769900)

In [ ]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5626863837242126),
 ('facebook', 0.5589815974235535),
 ('amazon', 0.4820205271244049),
 ('airbnb', 0.4768913984298706),
 ('disney', 0.4409243166446686),
 ('software', 0.43702954053878784),
 ('volkswagen', 0.436532586812973),
 ('toshiba', 0.4363655149936676),
 ('uber', 0.43553879857063293),
 ('fbi', 0.43474552035331726)]

**skip gram**

In [ ]:
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples = w2v_modelo_sg.corpus_count,
                 epochs = 30,
                 compute_loss = True,
                 callbacks=[callback()])

Loss após a época 0: 1697830.0
Loss após a época 1: 1359959.5
Loss após a época 2: 1160477.0
Loss após a época 3: 974708.0
Loss após a época 4: 983052.0
Loss após a época 5: 965750.0
Loss após a época 6: 809030.5
Loss após a época 7: 837879.0
Loss após a época 8: 719260.0
Loss após a época 9: 695912.0
Loss após a época 10: 673677.0
Loss após a época 11: 659187.0
Loss após a época 12: 679481.0
Loss após a época 13: 659922.0
Loss após a época 14: 567042.0
Loss após a época 15: 629479.0
Loss após a época 16: 615091.0
Loss após a época 17: 564709.0
Loss após a época 18: 586833.0
Loss após a época 19: 576479.0
Loss após a época 20: 487878.0
Loss após a época 21: 413270.0
Loss após a época 22: 405092.0
Loss após a época 23: 398776.0
Loss após a época 24: 395456.0
Loss após a época 25: 388448.0
Loss após a época 26: 354938.0
Loss após a época 27: 371028.0
Loss após a época 28: 366214.0
Loss após a época 29: 359072.0


(13208662, 14769900)

In [ ]:
w2v_modelo_sg.wv.most_similar("google")

[('apple', 0.43627840280532837),
 ('android', 0.4222179055213928),
 ('reguladores', 0.3971448540687561),
 ('yahoo', 0.38743484020233154),
 ('verizon', 0.3778606057167053),
 ('facebook', 0.37057068943977356),
 ('pfizer', 0.36726638674736023),
 ('automóveis', 0.3619340658187866),
 ('concorda', 0.35202541947364807),
 ('toshiba', 0.34956300258636475)]

In [ ]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_sg.txt", binary=False)

**Classificador**

In [ ]:
import spacy
nlp = spacy.load("pt_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])

def tokenizador(texto):
  tokens_validos = []
  doc = nlp(texto)

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos

texto = "Adoro a 342342 #$@#@#$ cidade de caldas novas!"
tokenizador(texto)

['adoro', 'cidade', 'caldas']

In [ ]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras, modelo):
  vetor_resultante = np.zeros(300)
  for pn in palavras:
    try:
      vetor_resultante =+ modelo.wv.get_vector(pn)
    except KeyError:
      pass

  return vetor_resultante

In [ ]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

  return matriz

In [ ]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

(90000, 300)
(20513, 300)
(90000, 300)
(20513, 300)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter = 800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return RL

In [ ]:
RL_cbow = classificador(w2v_modelo,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.59      0.37      0.46      6103
   cotidiano       0.33      0.56      0.42      1698
     esporte       0.69      0.60      0.64      4663
   ilustrada       0.04      0.56      0.07       131
     mercado       0.68      0.56      0.62      5867
       mundo       0.42      0.62      0.50      2051

    accuracy                           0.52     20513
   macro avg       0.46      0.55      0.45     20513
weighted avg       0.60      0.52      0.54     20513



In [ ]:
RL_cbow = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.63      0.36      0.46      6103
   cotidiano       0.34      0.58      0.43      1698
     esporte       0.74      0.59      0.66      4663
   ilustrada       0.04      0.63      0.08       131
     mercado       0.66      0.62      0.64      5867
       mundo       0.43      0.62      0.51      2051

    accuracy                           0.53     20513
   macro avg       0.47      0.57      0.46     20513
weighted avg       0.62      0.53      0.56     20513

